In [11]:
import zipfile
import os
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import defaultdict
import ast
from tqdm.notebook import tqdm as tqdm
import pickle

In [6]:
def helper(inv_idx_freq, inv_idx_postings, set_of_tokens, docID):
    for j in set_of_tokens:
        inv_idx_postings[j].append(docID)
        inv_idx_freq[j] = inv_idx_freq[j] + 1
        
        

In [7]:
inv_idx_freq  = defaultdict(int) # initializing the doc Frequency dictionary as a default dict which return 0 whenver a key does not exists.
inv_idx_postings  = defaultdict(list) # intializing the postings dictionary to return an empty list whenever it is invoked for a key that does not exits.
sample = 5

folder_path = 'Q1_2_output'
for filename in tqdm(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            contents = file.read()
            docID = int(filename[-4:])
            # print(docID)
            # set_of_tokens = set(contents.strip('][').split(','))
            set_of_tokens = set(ast.literal_eval(contents))
            helper(inv_idx_freq, inv_idx_postings, set_of_tokens, docID)
            # print(f'{filename}: {contents}')
            # print(set_of_tokens)
            # sample -= 1
            # if (sample == 0):
            #     break


#sort the index in alphabetical order wrt terms
inv_idx_postings = dict(sorted(inv_idx_postings.items()))
inv_idx_freq = dict(sorted(inv_idx_freq.items()))

for key in inv_idx_postings:
    inv_idx_postings[key].sort() #sort the indivdual posting lists correponging to each term in the index


  0%|          | 0/1400 [00:00<?, ?it/s]

In [13]:
pickle.dump(inv_idx_postings, open('inv_idx_postings.pkl', 'wb'))
pickle.dump(inv_idx_freq, open('inv_idx_freq.pkl', 'wb'))

In [14]:
inv_idx_postings = pickle.load(open("inv_idx_postings.pkl", 'rb'))
inv_idx_freq = pickle.load(open("inv_idx_freq.pkl", 'rb'))

In [ ]:
print(inv_idx_postings)


In [ ]:
print(inv_idx_freq)